In [13]:
import warnings

import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np

from joblib import load
from numpy import genfromtxt

import pickle
import pandas as pd
import os

In [20]:
# Import Model
def import_model():
    list_model = []
    path = './models'
    for i in os.listdir(path):
        list_model.append(i)
    print(list_model)
    return list_model

In [4]:
# Import normal Dataset
def import_dataset():
    with open('./Dataset/csv/Attack_merge.csv', newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        y = rows[10]
        x = rows.drop([10], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2)
    return X_train, X_test, y_train, y_test

In [98]:
import itertools
def combinations(list_model, num):
    return [x for x in itertools.combinations(list_model, num)]

In [101]:
combination_model = combinations(['CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib', 'LSTM.h5', 'RF.joblib', 'SVM.joblib', 'XGB.joblib'], 3)
print(type(combination_model[0]))

<class 'tuple'>


In [116]:
def kappa_statistics(pred_labels, y_test):
    ks = 0
    length = len(pred_labels)
    for n in range(len(pred_labels[0])):
        # value_b = ''
        for o in range(0, length):
            if o == 0:
                continue
            elif pred_labels[o][n] != pred_labels[o-1][n]:
                break
            elif o == length-1:
                ks += 1
            else:
                continue
    ks_percentage = ks/len(y_test) * 100
    #print("KAPPA STATS: " + str(ks_percentage))

    return ks_percentage

In [117]:
def double_fault(pred_labels, y_test):
    wrong_gt = ([l for l, m in enumerate(y_test) if m != pred_labels[0][l]])
    df = 0
    length = len(pred_labels)
    for n, o in enumerate(wrong_gt):
        for p in range(0, length):
            if p == 0:
                continue
            elif pred_labels[p][o] != pred_labels[p-1][o]:
                break
            elif p == length-1:
                df += 1
            else:
                continue
    df_percentage = df/len(y_test) * 100
    #print("DOUBLE FAULT: " + str(df_percentage))
    
    return df_percentage

In [156]:
def voting_average(good_team, f1_score_team):
    best_team_index = 0
    best_f1_score = 0
    for i in range(len(f1_score_team)):
        if np.max(f1_score_team[i]) > best_f1_score:
            best_f1_score = np.max(f1_score_team[i])
            best_team_index = i

    return good_team[best_team_index], best_f1_score

In [158]:
def ensemble_learning(num_teams):
    good_team = []
    f1_score_team = []
    path = "./models/"
    list_model = import_model()
    X_train, X_test, y_train, y_test = import_dataset()
    y_train_ex = tf.expand_dims(y_train, axis=1)
    X_train_ex = tf.expand_dims(X_train, axis=2)
    y_test_ex = tf.expand_dims(y_test, axis=1)
    X_test_ex = tf.expand_dims(X_test, axis=2)

    combination_model = combinations(list_model=list_model, num = num_teams)
    # print(combination_model)
    for j in range(0, len(combination_model)):
        pred_labels = []
        f1_inteam =[]
        for k in range(0, len(combination_model[j])):
            if combination_model[j][k] == "CNN.h5" or combination_model[j][k] == "LSTM.h5":
                prediction = tf.keras.models.load_model(path + combination_model[j][k]).predict(X_test_ex)
                pred_label = np.rint(prediction)
                f1_inteam.append(sklearn.metrics.f1_score(y_test_ex, pred_label, average="binary"))

            elif combination_model[j][k] == "DNN4.h5":
                prediction = tf.keras.models.load_model(path + combination_model[j][k]).predict(X_test)
                pred_label = np.rint(prediction)
                f1_inteam.append(sklearn.metrics.f1_score(y_test, pred_label[0:,1], average="binary"))
                prediction = prediction[0:,1]

            else:
                prediction = load(path + combination_model[j][k]).predict(X_test)
                pred_label = np.rint(prediction)
                f1_inteam.append(sklearn.metrics.f1_score(y_test, pred_label, average="binary"))

            pred_labels.append(np.rint(prediction))
        #print(pred_labels)
        # print(combination_model[j])
        # print(f1_inteam)
        ks_percentage = kappa_statistics(pred_labels, y_test)
        df_percentage = double_fault(pred_labels, y_test)
        
        if ks_percentage > 50 and ks_percentage < 91 and df_percentage < 50:
            good_team.append(combination_model[j])
            f1_score_team.append(f1_inteam)
            
    best_team, best_team_f1 = voting_average(good_team, f1_score_team)
    print("best_team, best_team_f1", best_team, best_team_f1)
    



In [159]:
ensemble_learning(2)

['CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib', 'LSTM.h5', 'RF.joblib', 'SVM.joblib', 'XGB.joblib']


c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgb

best_team, best_team_f1 ('DT.joblib', 'LR.joblib') 0.9856364308147054


In [160]:
ensemble_learning(3)

['CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib', 'LSTM.h5', 'RF.joblib', 'SVM.joblib', 'XGB.joblib']


c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgb

best_team, best_team_f1 ('DNN4.h5', 'DT.joblib', 'LR.joblib') 0.9861114299488074


c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [161]:
ensemble_learning(5)

['CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib', 'LSTM.h5', 'RF.joblib', 'SVM.joblib', 'XGB.joblib']


c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgb

best_team, best_team_f1 ('CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib') 0.9864575258766131
